In [1]:
# Feito por
# Leonardo Heidi Almeida Murakami, 11260186
# Vítor Neri Roque, 12558099

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv")

In [4]:
#CONSTANTS
STATE = 'PA' #@param {type:"string"}
FIRST_PEAK_DAY_CUT =  130#@param {type: "integer"}
HYPEROPT_EPOCHS = 210000 #@param {type:"slider", min:10000, max:20000000, step:200000}
MOVING_AVERAGE_INTERVAL = 7 #@param {type: "integer"}

# Data Pre-processing
Filter data by state, since we only want data from Pará

In [5]:
df_pa = df.query(f"state == '{STATE}'").reset_index(drop=True).fillna(0)
pop = (df_pa['totalCases'].iat[-1]*1e5/(df_pa['totalCases_per_100k_inhabitants'].iat[-1]))


In [6]:
df_pa['date'] = pd.to_datetime(df_pa['date'])
rolling_df_pa = df_pa.copy()
rolling_df_pa.index = rolling_df_pa['date']

# Calculate cumulative cases
rolling_df_pa['sumCases'] = rolling_df_pa.newCases.cumsum()
rolling_df_pa['suspected'] = rolling_df_pa.sumCases.apply(lambda x: pop - x)
rolling_df_pa['infected'] = rolling_df_pa['totalCases'] - rolling_df_pa['recovered']
numeric_columns = rolling_df_pa.select_dtypes(include=['number']).columns
moving_average_df = rolling_df_pa[numeric_columns].rolling(MOVING_AVERAGE_INTERVAL).mean().dropna()
moving_average_df = moving_average_df.reset_index()

## Comparison between Moving Average (top) and Raw Data (bottom)

In [7]:
x_col = 'index'
y_col = 'newCases'
fig = px.line(rolling_df_pa.reset_index(drop=True).reset_index(), x=x_col, y=y_col)
fig.add_scatter(x=moving_average_df.reset_index()[x_col], y=moving_average_df.reset_index()[y_col])

fig.data[0].name = "Original Data"
fig.data[0].showlegend = True
fig.data[1].name = f"{MOVING_AVERAGE_INTERVAL} Day Moving Average"
fig.data[1].showlegend = True

fig.show()

Como queremos o primeiro pico, cortaremos parte dos dados fora

In [8]:
chosen_moving_average = moving_average_df
first_peak_day_cut = FIRST_PEAK_DAY_CUT

rolling_df_pa = chosen_moving_average.iloc[:first_peak_day_cut, :]
fig = px.line(rolling_df_pa.reset_index(), x='date', y='newCases')
fig.show()

## SIR Model



In [9]:
class SIRModel:
  def __init__(self, b, k, total_population):
    # SIR Params
    self.N = total_population #pop size

    self.S = [] #suspected proportion
    self.I = [] #infected proportion
    self.R = [] #recovered proportion

    self.dS = [] #delta suspected proportion
    self.dI = [] #delta infected proportion
    self.dR = [] #delta recovered proportion
    # Hyperparams
    self.b = b # in [0, 5]
    self.k = k # in [0.067, 0.2]

  def euler_method_new_tuple(self, interval):
    s, i, r = self.S[-1], self.I[-1], self.R[-1]
    # based on: https://www.maa.org/press/periodicals/loci/joma/the-sir-model-for-spread-of-disease-eulers-method-for-systems
    s_n = s - (self.b * s * i * interval)
    i_n = i + (((self.b * s) - self.k) * i)*interval
    r_n = r + (self.k * i)*interval

    ds_n = -((self.b * s) * i)*interval
    di_n = (((self.b * s) - self.k) * i)*interval
    dr_n = (self.k * i)*interval
    return ((s_n, i_n, r_n), (ds_n, di_n, dr_n))

  def fit(
      self, 
      data, 
      cases_column='newCases', 
      suspected_column='suspected', 
      recovered_column='recovered', 
      fit_until=None,
      metricI=True, 
      metricS=True, 
      metricR=True,
      interval=1
    ):
    starting_point = 0
    self.I.append(data.loc[starting_point, cases_column]/self.N)
    self.S.append(1 - self.I[0])
    self.R.append(0)
    if fit_until is None:
      fit_until = len(data) - 1
    for j in range(starting_point, fit_until, interval):
      #update sir value
      ((s, i, r), (ds, di, dr)) = self.euler_method_new_tuple(j-starting_point)
      self.S.append(s)
      self.I.append(i)
      self.R.append(r)
      
      self.dS.append(ds)
      self.dI.append(di)
      self.dR.append(dr)

    self.S = np.array(self.S)
    self.I = np.array(self.I)
    self.R = np.array(self.R)

    self.dS = np.array(self.dS)
    self.dI = np.array(self.dI)
    self.dR = np.array(self.dR)

  def score(self, y_true_I, y_true_S, y_true_R, metricI=True, metricS=True, metricR=True):
    diffI = (self.I - y_true_I[:len(self.I)]/self.N)**2 if metricI else np.array([0])
    diffS = (self.S - y_true_S[:len(self.S)]/self.N)**2 if metricS else np.array([0])
    diffR = (self.R - y_true_R[:len(self.R)]/self.N)**2 if metricR else np.array([0])
    return sum(diffI + diffS + diffR)
  
    

In [11]:
import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm

b_space = np.linspace(0, 5, 20000)
k_space = np.linspace(1/15, 1/5, 1000)
params_space = np.array(np.meshgrid(b_space, k_space)).T.reshape(-1,2)

best = {
    "b": 0,
    "k": 0,
    "metric": np.inf
}

In [12]:
hyperopt_epochs = HYPEROPT_EPOCHS
interval = 1
print(f"Testing {hyperopt_epochs/len(params_space)*100}% of our params space ({hyperopt_epochs} out of {len(params_space)}) possible combinations")
for _ in tqdm(range(hyperopt_epochs)):
    idx = np.random.randint(0, len(params_space))
    b, k = params_space[idx, 0], params_space[idx, 1]
    model = SIRModel(b, k, pop)
    model.fit(rolling_df_pa, fit_until=first_peak_day_cut-1, interval=interval)
    score = sum(((model.dI + model.dR) - rolling_df_pa['newCases'][range(0, first_peak_day_cut-1, interval)]/model.N)**2)
    if score < best["metric"]:
      best['b'] = b
      best['k'] = k
      best['metric'] = score

Testing 1.05% of our params space (210000 out of 20000000) possible combinations


100%|██████████| 210000/210000 [01:20<00:00, 2609.35it/s]


In [13]:
print("Best parameters found!")
print(f"b:\t{best['b']}")
print(f"k:\t{best['k']}")

Best parameters found!
b:	0.20001000050002501
k:	0.1989322655989323


In [14]:
model = SIRModel(best['b'], best['k'], pop)
model.fit(moving_average_df)

In [15]:
y_col = 'infected' #@param ["suspected", "infected", "recovered"]

y_col_real = {
    "suspected": "suspected",
    "infected": "newCases",
    "recovered": "recovered"
}
y_col_sir = {
    "suspected": model.S,
    "infected": (-model.dS),
    "recovered": model.R
}
fig = px.line(x=rolling_df_pa['date'], y=rolling_df_pa[y_col_real[y_col]], title='SIR Model x True Infected')
fig.add_scatter(x=rolling_df_pa['date'][range(0, first_peak_day_cut-1, interval)], y=np.array(y_col_sir[y_col])*model.N)

fig.data[0].name = "SIR"
fig.data[0].showlegend = True
fig.data[1].name = "Real Data"
fig.show()

In [16]:
y_col = 'infected' #@param ["suspected", "infected", "recovered"]

y_col_real = {
    "suspected": "suspected",
    "infected": "newCases",
    "recovered": "recovered"
}
y_col_sir = {
    "suspected": model.S,
    "infected": (-model.dS),
    "recovered": model.R
}
fig = px.line(x=moving_average_df['date'], y=moving_average_df[y_col_real[y_col]], title=f'SIR Model x True {y_col}')
fig.add_scatter(x=moving_average_df['date'], y=np.array(y_col_sir[y_col])*model.N)

fig.data[0].name = "Real Data"
fig.data[0].showlegend = True
fig.data[1].name = "SIR"
fig.show()

In [17]:
y_col = 'suspected' #@param ["suspected", "infected", "recovered"]

y_col_real = {
    "suspected": "suspected",
    "infected": "newCases",
    "recovered": "recovered"
}
y_col_sir = {
    "suspected": model.S,
    "infected": (-model.dS),
    "recovered": model.R
}
fig = px.line(x=moving_average_df['date'][:FIRST_PEAK_DAY_CUT], y=moving_average_df[y_col_real[y_col]][:FIRST_PEAK_DAY_CUT], title=f'SIR Model x True {y_col}')
fig.add_scatter(x=moving_average_df['date'][:FIRST_PEAK_DAY_CUT], y=np.array(y_col_sir[y_col][:FIRST_PEAK_DAY_CUT])*model.N)

fig.data[0].name = "Real Data"
fig.data[0].showlegend = True
fig.data[1].name = "SIR"
fig.show()

In [18]:
y_col = 'recovered' #@param ["suspected", "infected", "recovered"]

y_col_real = {
    "suspected": "suspected",
    "infected": "newCases",
    "recovered": "recovered"
}
y_col_sir = {
    "suspected": model.S,
    "infected": (-model.dS),
    "recovered": model.R
}
fig = px.line(x=moving_average_df['date'][:FIRST_PEAK_DAY_CUT], y=moving_average_df[y_col_real[y_col]][:FIRST_PEAK_DAY_CUT], title=f'SIR Model x True {y_col}')
fig.add_scatter(x=moving_average_df['date'][:FIRST_PEAK_DAY_CUT], y=np.array(y_col_sir[y_col][:FIRST_PEAK_DAY_CUT])*model.N)

fig.data[0].name = "Real Data"
fig.data[0].showlegend = True
fig.data[1].name = "SIR"
fig.show()